# Converting PSSE data from MATLAB to Numpy
For now, we will prepare the $A$ matrices and ground truth (nonnoisy) measurements $b$ vector using MATLAB. Then, we will convert these variables to numpy arrays. Note that $x_{sol}$ is the power flow solution, and $b_i = x_{sol}^TA_ix_{sol}$.

## Order of the measurements 
Note that $x_{sol} \in \mathbb{R}^{n} $, 
$b \in \mathbb{R}^{m} $, and 
$A = [vec(A_i)] \in \mathbb{R}^{n\times m}$, where $n = 2n_{bus} -1$ as we set the imaginary component of the slack bus voltage to zero.

Consider the following order for measurement types:
$$[p_{i}  ,q_{i}, v_i^2, p_{f}, q_{f}, p_{t},  q_{t}]$$
Since we are starting from pf measurements, we put the PF measurements first in $A$. The rest of the order is the same with PF measurements removed:
$$\text{[PF measurements, rest of the measurements]},$$
where PF measurements are:
$$\text{[$p_i$ for pv and pq buses, $q_i$ for pq buses, $v_i^2$ for pv and ref buses]}.$$

Note that $M_{pf}$ is the number of pf measurements, where $M_{pf} = 2N_{pv} + 2N_{pq} +1 = 2N_{bus} -1$.

You may not need the following data, but they will be provided in case you do. 
- $pv, pq, ref$: give the list of which buses are PV, PQ, and buses.
- $line$: Each row gives the from and to bus number.

Change ```"case14.mat"``` with your file in the following code. 

In [6]:
import numpy as np
import scipy.io as sio          # SciPy ≥ 1.10 recommended
from scipy import sparse

# --- 1.  Load the .mat file --------------------------------------------------
mat = sio.loadmat(
    "case1354pegase.mat",
    squeeze_me=True,            # drop singleton dimensions
    simplify_cells=True         # unwrap cell arrays / structs automatically
)                               # spmatrix=True by default, so sparse MATLAB
                                # variables come back as SciPy sparse objects:contentReference[oaicite:0]{index=0}

# --- 2.  Grab the variables you need -----------------------------------------
A   = mat["A"]      # sparse matrix
line = mat["line"]
bsol   = np.asarray(mat["b"]).ravel()       # 1-D NumPy vector
xsol = np.asarray(mat["xsol"]).ravel()  
pv   = np.asarray(mat["pv"]).ravel()
pq   = np.asarray(mat["pq"]).ravel()
ref  = mat["ref"]

# --- 3.  (Optional) tidy-up / conversions ------------------------------------
# Make sure A is the actual sparse matrix (not wrapped in a 1×1 object array)
if isinstance(A, np.ndarray) and A.dtype == object:
    print('Problem')
    A = A.item()                # unwrap cell / struct entry:contentReference[oaicite:1]{index=1}


# If you really need dense NumPy arrays:
# A_dense = A.toarray()           # beware of memory blow-up for large matrices


In [7]:
line

array([[1074,  802],
       [ 652, 1103],
       [ 856,  592],
       ...,
       [ 690, 1273],
       [ 690, 1273],
       [ 429,  639]], dtype=uint16)

In [4]:
bsol.shape

(10,)

In [5]:
xsol

array([ 1.        ,  0.80643107, -0.19      ])